In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import argparse
from torch.utils.tensorboard import SummaryWriter

from dataset import load_data, GtzanDataset
from baseline_cnn import CNN

In [3]:
class Config():
    def __init__(self):
        self.batch_size = 32
        self.epochs = 5
        self.lr = 0.01
args = Config()

In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
is_test = True

dev_reporter = Reporter('baseline', args.epochs)

# Load Data
GTZAN = load_data(is_test=is_test)

train_dataset = GtzanDataset(GTZAN.train_x, GTZAN.train_y, train=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size)

test_dataset = GtzanDataset(GTZAN.test_x, GTZAN.test_y, train=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=args.batch_size, shuffle=True)

# Initialize network
model = CNN('yeet', get_report_data=True).to(device)

dev_reporter.record_first_batch(model, train_loader)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)
# Train Network
for epoch in range(args.epochs):
    print('epoch num ', epoch)
    dev_reporter.reset_epoch_data()
   
    for batch_idx, (data, targets) in tqdm(enumerate(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        preds = model(data)
        loss = criterion(preds, targets)

        dev_reporter.record_batch_data(preds, targets, loss)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    dev_reporter.record_epoch_data(model, epoch)
    

folder exists lad
keeping log
recording first batch data
train set has:  7
0it [00:00, ?it/s]epoch num  0
7it [00:42,  6.05s/it]
0it [00:00, ?it/s]epoch num  1
7it [00:41,  5.96s/it]
0it [00:00, ?it/s]epoch num  2
7it [00:41,  5.93s/it]
0it [00:00, ?it/s]epoch num  3
7it [00:41,  5.95s/it]
0it [00:00, ?it/s]epoch num  4
7it [00:41,  5.92s/it]


In [68]:
dev_reporter.set_post_training_values(model, train_dataset, test_dataset)
dev_reporter.report_on_model()

  0%|          | 0/4 [00:00<?, ?it/s]
\Getting report on model...
100%|██████████| 1/1 [00:00<00:00,  6.11it/s]


(11, 0)

In [65]:
from sklearn.metrics import confusion_matrix
from reporter_utils import *
import torchvision
import os, re, os.path

possible_names = ['baseline', 'segmented']

class Reporter():
    def __init__(self, name, max_epochs):
        if name not in possible_names:
            raise ValueError('name is not recognized from possible experiment names')
        
        my_path = os.path.join(os.getcwd(), 'reports', name)

        if os.path.exists(my_path):
            print('folder exists lad')
            for root, dirs, files in os.walk(my_path):
                for file in files:
                    os.remove(os.path.join(root, file))

        self.name = name
        self.log_path = 'reports/' + name + '/train'
        self.max_epochs = max_epochs

        self.train_confusion_matrix = None
        self.test_confusion_matrix = None
        self.train_summary_writer = SummaryWriter(self.log_path)
        # self.test_summary_writer = SummaryWriter('logs/tensorboard/' + name + '/test')

    def set_post_training_values(self, model, train_set, test_set):
        self.model = model
        self.train_set = train_set
        self.test_set = test_set
    def report_on_model(self):
        print('\n\Getting report on model...')
        train_loader = torch.utils.data.DataLoader(self.train_set, batch_size=64)
        test_loader = torch.utils.data.DataLoader(self.test_set, batch_size=64)
        
        self.train_predictions = get_all_preds(self.model, train_loader)
        self.test_predictions = get_all_preds(self.model, test_loader)

        train_num_correct = get_num_correct(self.train_predictions, self.train_set.targets)
        test_num_correct = get_num_correct(self.test_predictions, self.test_set.targets)
        
        self.train_confusion_matrix = confusion_matrix(
                                    self.train_set.targets,
                                    self.train_predictions.argmax(dim=1))
        self.test_confusion_matrix = confusion_matrix(
                                    self.test_set.targets,
                                    self.test_predictions.argmax(dim=1))
        
        return (train_num_correct, test_num_correct)
                                    
    
    def show_confusion_matrix(self, train=True):
        if self.confusion_matrix is not None:
            if train:
                plot_confusion_matrix(self.train_confusion_matrix)
            else:
                plot_confusion_matrix(self.test_confusion_matrix)
        else:
            raise ValueError(
                'confusion matrix is not generated yet, please run Reporter.report_on_model() to generate it.')
    
    def record_first_batch(self, model, loader):
        print('recording first batch data')
        with torch.no_grad():
            images, labels = next(iter(loader))
            first_spectrogram = images.unsqueeze(1)[0,:,:,:]
            self.train_set_len = len(loader)
            print('train set has: ', self.train_set_len)
            self.train_summary_writer.add_image('images', first_spectrogram)
            self.train_summary_writer.add_graph(model, first_spectrogram)
    def reset_epoch_data(self):
        self.total_loss = 0
        self.total_correct = 0
    def record_batch_data(self, predictions, targets, loss): # could be further extended
        self.total_loss += loss.item()
        self.total_correct += get_num_correct(predictions, targets.numpy())
    def record_epoch_data(self, model, epoch):
        self.train_summary_writer.add_scalar("Loss", self.total_loss, epoch)
        self.train_summary_writer.add_scalar("Correct", self.total_correct, epoch)
        self.train_summary_writer.add_scalar("Accuracy", self.total_correct / self.train_set_len, epoch)

        self.train_summary_writer.add_histogram("conv1.bias", model.conv1.bias, epoch)
        self.train_summary_writer.add_histogram("conv1.weight", model.conv1.weight, epoch)
        self.train_summary_writer.add_histogram("conv2.bias", model.conv2.bias, epoch)
        self.train_summary_writer.add_histogram("conv2.weight", model.conv2.weight, epoch)
        if (epoch-1) == self.max_epochs:
            print('train summary writer is closing')
            self.train_summary_writer.close()

In [31]:
reporter = Reporter('baseline')

In [32]:
reporter.set_post_training_values(model, train_dataset, test_dataset)


In [34]:
reporter.report_on_model()

  0%|          | 0/4 [00:00<?, ?it/s]

Getting correct predictions...
100%|██████████| 1/1 [00:00<00:00,  6.22it/s]train_num_correct 33
test_num_correct 0



In [1]:
# type()
reporter.confusion_matrix.shape

NameError: name 'reporter' is not defined